In [ ]:
%matplotlib notebook

%load_ext autoreload
%autoreload 2

#Import Libraries
import tensorflow as tf
import numpy as np
import math
import os
import matplotlib
import matplotlib.pyplot as plt
import datetime
import time

import keras
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.layers import LSTM
from keras.utils.vis_utils import plot_model

from sklearn.metrics import recall_score, precision_score
from models import save_model, generate_model_CNN, plot_performance
from generators import generator, decode

In [ ]:
X_load = np.load('../data/processed_data/m4015_session2/X_serial.npy')
Y_load = np.load('../data/processed_data/m4015_session2/y_serial.npy')

res = tf.one_hot(Y_load, depth=2)
with tf.Session() as sess:
    Y_load = (sess.run(res))

split_index = int(X_load.shape[0]*0.8)
    
X_train = X_load[:split_index]
Y_train = Y_load[:split_index]
X_val = X_load[split_index:]
Y_val = Y_load[split_index:]


class_weight = {0: 1., 1: 50.,}
predict_early = 50

In [ ]:
plt.figure()
plt.plot(X_load)
plt.plot(np.argmax(Y_load, axis=1))
plt.show()

In [ ]:
batch_size = 1000
n_hidden = 100
n_dim = 1
input_shape = (n_hidden, n_dim)


model = generate_model_CNN(input_shape)

print(model.summary())

In [ ]:
train_generator = generator(X_train, Y_train, batch_size, n_hidden, predict_early=predict_early)
validate_generator = generator(X_val, Y_val, batch_size, n_hidden, predict_early=predict_early)

history = model.fit_generator(train_generator,
                              steps_per_epoch=1000,
                              epochs=20, 
                              verbose=1,
                              validation_data=validate_generator,
                              validation_steps=1000,
                              class_weight=class_weight, )

save_model(model, name=predict_early)

In [ ]:
plot_performance(history)

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from models import make_accuracy_matrix_plot

In [ ]:
validate_generator = generator(X_val, Y_val, batch_size*10, n_hidden)

# make_accuracy_matrix_plot(model, validate_generator)
make_accuracy_matrix_plot(model, validate_generator, ref='est')

In [ ]:
save_dir = 'plots'
if not os.path.exists(save_dir):
        os.makedirs(save_dir)

second_dir = 'simple'
save_dir = os.path.join(save_dir, second_dir)  
if not os.path.exists(save_dir):
        os.makedirs(save_dir)




In [ ]:
X = X_val
y = np.argmax(Y_val, axis=1)
t = np.arange(X.shape[0])

n_slices = X.shape[0] - n_hidden

all_y = np.zeros((n_slices, 2))

for i in range(n_slices):
    x_input = np.reshape(X[i:i+n_hidden], (1, n_hidden, 1))
    all_y[i] = model.predict(x_input)
    


In [ ]:
y_pred_int = np.argmax(all_y, axis=1)

mask = all_y[:,1]>0.99
y_pred_int = np.array(mask, dtype=int)

y_res = np.zeros_like(t)
y_res[int(n_hidden/2):int(-n_hidden/2)] = y_pred_int

In [ ]:
y_scale = np.zeros_like(t)
y_scale[int(n_hidden/2):int(-n_hidden/2)] = all_y[:,1]

y.shape, y_scale.shape

In [ ]:
%matplotlib notebook

plt.figure()
plt.plot(t, X, label='LFP')
plt.plot(t, y-3, alpha=0.5, label='manual scoring')
plt.plot(t, y_res-2, alpha=0.5, label='CNN')
plt.plot(t, y_scale)
plt.xlim(150000, None)
plt.ylim(None, 1)
plt.legend(framealpha=0)
plt.savefig('plots/nice.pdf')
plt.show()

In [ ]:
y_scale[int(n_hidden/2):int(-n_hidden/2)] = all_y[:,1]

In [ ]:
y_scale[int(n_hidden/2):int(-n_hidden/2)]

In [ ]:
plt.figure()
plt.hist(all_y[:,1], bins=100, cumulative=-1, density=True)
plt.show()

In [ ]:
all_y

In [ ]:
X_load = np.load('../data/processed_data/m4015_session2/X_serial.npy')
Y_load = np.load('../data/processed_data/m4015_session2/y_serial.npy')

In [ ]:
X_load.shape

In [ ]:
Y_load.shape

In [ ]:
plt.figure()
plt.plot(Y_load)
plt.show()